<div style="text-align:right">
    <b>Author:</b> Andreas P. Koenzen (akc at apkc.net) / <a href="http://www.apkc.net">www.apkc.net</a>
</div>

# Linear Regression

## Imports

In [ ]:
%run '../../../../imports.py'

## Configuration

In [ ]:
%run '../../../../config.py'

## Environment variables

In [ ]:
%run '../../../../env_variables.py'

## Salary-Experience example
- E = Experience in years. ($X$ = Predictor or Feature or Independent Variable)
- S = Salary in thousands of CAD. ($Y$ = Response or Outcome or Dependent Variable) Not to be confused with the Predicted Value $\hat{Y}$.

The data for this notebook is artificially generated by sampling from a Uniform distribution. Use the Gaussian distribution to sample values for C to compute the value of Y in order to introduce more variation.

In [ ]:
# generate the data
rng = np.random.RandomState(1) # seed the random function to obtain the same numbers
sample_number = 10

# load data into X and Y
X = 10 * (rng.rand(sample_number) + 0.1)
Y = 10 * (1.0 * X + 5.0 + rng.rand(sample_number))

# convert the data to integers
X = X.astype(int)
Y = Y.astype(int)

# plot the data
plt.scatter(X, Y)
plt.grid(True)
plt.xlabel('Years of Experience')
plt.ylabel('Salary')

In [ ]:
print('Values:')
data = pd.DataFrame({'X': X, 'Y': Y})
data

### Compute the weights of the line

Do N iterations and compute the error.

In [ ]:
debug = False
iterations = 500
kappa = 0.1

# initialize vector w to all 1.0
w = np.array([[1.0, 1.0]])
# w
E = 0

for k in range(0, iterations):
    # build the x vector. remember that it is the dummy value and the value of feature N.
    # in this case it should be a 2D vector.
    x0 = np.ones((len(X), 1), dtype=float)
    x = np.hstack([x0, X[:, np.newaxis]])
    if k + 1 == iterations and debug:
        x
    
    y = Y[:, np.newaxis]
    
    # compute the Mean Square Error
    E = (1 / (2 * len(X))) * np.sum(((y - (x @ w.T)) ** 2), axis=0, keepdims=True)
    E = np.asscalar(E)
    if k + 1 == iterations and debug:
        E
    if (k + 1) % 50 == 0:
        print("Iteration: {0} => Error: {1}".format(k + 1, E))
        
    # re-compute the weights using gradient descent
    g = (1 / len(X)) * np.sum(((y - (x @ w.T)) * x), axis=0, keepdims=True)
    if k + 1 == iterations and debug:
        g
    
    # update the weights
    w = w + (kappa * g)
    
w
E

## Plot the line

In [ ]:
# plot the data
plt.scatter(X, Y)
plt.grid(True)
plt.xlabel('Years of Experience')
plt.ylabel('Salary')

# plot line equation
plt.plot(
    range(1, 9),
    [(k * np.squeeze(w)[1] + np.squeeze(w)[0]) for k in range(1, 9)],
    c= "red"
);

***

In [ ]:
# Clear defined variables.
%reset

***
# Using scikit-learn

## Chapter 5: Machine Learning

### Python Data Science / Page 390

### Note
- Linear regression using Python's **scikit-learn** library.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from IPython.display import Math
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [ ]:
# generate the data
rng = np.random.RandomState(1) # generate a 1-D array of random numbers

# load data into X and Y
x = 10 * rng.rand(50) # generate 50 uniform random numbers [0,1)
y = 2 * x - 5 + rng.randn(50) # compute function y = f(x) = 2x - 5 + C, where C is a normal random number N(0,1)

# plot the data
plt.scatter(x, y)
plt.grid(True)

## Linear regression model formula
$\hat{y} = b + w_1 x_1 + ... + w_n x_n$, where:

1. $b$ = Bias / same as y-intercept or $b_0$
1. $w_1$ = weight of feature 1 / same as m (slope) or b1
1. $x_1$ = feature 1 or known input
1. $\hat{y}$ = predicted label or desired output

In [ ]:
model = LinearRegression(fit_intercept=True) # fit_intercept=True calculate Y-intercept
# generate the linear regression model
model.fit(x[:, np.newaxis], y)
print("Feature 1 slope (w1):", model.coef_[0])
print("Model intercept (w0):", model.intercept_)

# debug
print("")
print("Model fitting parameters:")
print("X Data Matrix {}".format(np.shape(x[:, np.newaxis])))
print("Y Data Matrix {}".format(np.shape(y)))
print("")

xfit = np.linspace(0, 10, 1000) # 1000 evenly spaced numbers [0,10]
yfit = model.predict(xfit[:, np.newaxis]) # make the predictions based on the model generated previously

# generate both plots: scatter and line
plt.scatter(x, y)
plt.plot(xfit, yfit)
plt.grid(True)

In [ ]:
# make an individual prediction based on the model
prediction = model.predict(np.array([[12]]))[0]
print("Individual Prediction:")
print("For x=12, ŷ={}".format(prediction))

xfit = np.linspace(0, 12, 1000)
yfit = model.predict(xfit[:, np.newaxis])

# generate scatter plot
plt.scatter(np.append(x, [12]), np.append(y, prediction))
plt.plot(xfit, yfit)
plt.grid(True)

# Notes

## Least Squares Criterion
A line that fits the data "best" will be one for which the n prediction errors — one for each observed data point — are as small as possible in some overall sense. The “Least Squares Criterion” which says to “minimize the sum of the squared prediction errors.” That is:

1. The equation of the best fitting line is: $\hat{y_i} = b_0 + b_1x_i$
2. We just need to find the values $b_0$ and $b_1$ that make the sum of the squared prediction errors the smallest it can be.
3. That is, we need to find the values $b_0$ and $b_1$ that minimize:

$$Q = \sum_{i=1}^{n} (y_i - \hat{y})^2$$

The goal is to reduce Q as much as we can.

## Nomenclature
1. $b_0$ or $b$: y-intercept (value of y when all the predictors equal zero)
1. $w_n$: weight or slope of feature n
1. $e_i$: i-th prediction error, equal to $y_i - \hat{y_{i}}$
1. MSE (“mean square error”): mean square prediction error (or residual error)
1. $x$: a predictor, explanatory, or independent variable in a linear regression model
1. $\bar{x}$: sample mean of x
1. $y$: the response, outcome, or dependent variable in a linear regression model
1. $\bar{y}$: sample mean of y (ignoring any predictors)
1. $\hat{y}$: predicted or fitted value of y in a linear regression model

***
# End